In [8]:
pip install --upgrade scikit-learn

Importation des librairies : 

In [14]:
import os
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
import joblib

In [10]:
# Charger les données
chemin_dossier = "C:/Users/paulm/Documents/GitHub/Projet7/data/"
df = pd.read_csv(os.path.join(chemin_dossier, 'processed_data.csv'))

# Sélectionner les colonnes catégorielles
colonnes_categorielles = df.select_dtypes(include='object').columns.tolist()

# Afficher le nombre de valeurs uniques dans chaque colonne catégorielle
for col in colonnes_categorielles:
    nb_valeurs_uniques = df[col].nunique()
    print(f"Colonne '{col}' contient {nb_valeurs_uniques} valeurs uniques.")

Colonne 'NAME_CONTRACT_TYPE' contient 2 valeurs uniques.
Colonne 'CODE_GENDER' contient 3 valeurs uniques.
Colonne 'FLAG_OWN_CAR' contient 2 valeurs uniques.
Colonne 'FLAG_OWN_REALTY' contient 2 valeurs uniques.
Colonne 'NAME_TYPE_SUITE' contient 7 valeurs uniques.
Colonne 'NAME_INCOME_TYPE' contient 8 valeurs uniques.
Colonne 'NAME_EDUCATION_TYPE' contient 5 valeurs uniques.
Colonne 'NAME_FAMILY_STATUS' contient 6 valeurs uniques.
Colonne 'NAME_HOUSING_TYPE' contient 6 valeurs uniques.
Colonne 'OCCUPATION_TYPE' contient 18 valeurs uniques.
Colonne 'WEEKDAY_APPR_PROCESS_START' contient 7 valeurs uniques.
Colonne 'ORGANIZATION_TYPE' contient 58 valeurs uniques.
Colonne 'FONDKAPREMONT_MODE' contient 4 valeurs uniques.
Colonne 'HOUSETYPE_MODE' contient 3 valeurs uniques.
Colonne 'WALLSMATERIAL_MODE' contient 7 valeurs uniques.
Colonne 'EMERGENCYSTATE_MODE' contient 2 valeurs uniques.


In [11]:
# Supprimer les colonnes avec trop de valeurs uniques
colonnes_a_supprimer = ['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']
df.drop(columns=colonnes_a_supprimer, inplace=True)

In [12]:
# Sélectionner uniquement les colonnes numériques
colonnes_numeriques = df.select_dtypes(include=[np.number])

# Identifier les colonnes contenant des valeurs infinies
colonnes_infinies = colonnes_numeriques.columns.to_series()[np.isinf(colonnes_numeriques).any()]

# Affichage des colonnes avec des valeurs infinies
print("Colonnes avec des valeurs infinies :")
print(colonnes_infinies)

# Identifier les lignes contenant des valeurs infinies
lignes_avec_infinis = df.index[np.isinf(colonnes_numeriques).any(axis=1)]

# Affichage des indices des lignes contenant des valeurs infinies
print("\nIndices des lignes avec des valeurs infinies :")
print(lignes_avec_infinis.tolist())

# Suppression des lignes contenant des valeurs infinies
df.drop(lignes_avec_infinis, inplace=True)

# Sélection des colonnes numériques après la suppression des lignes
colonnes_numeriques = df.select_dtypes(include=[np.number])

# Vérification
print("\nReste-t-il des valeurs infinies ?")
reste_infinis = np.isinf(colonnes_numeriques).any().any()
print(reste_infinis)

# Si des valeurs infinies restent, les remplacer par NaN
if reste_infinis:
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    print("\nLes valeurs infinies ont été remplacées par NaN.")

# Vérification
reste_infinis = np.isinf(colonnes_numeriques).any().any()
print("\nReste-t-il des valeurs infinies après remplacement ?")
print(reste_infinis)

Colonnes avec des valeurs infinies :
PREV_APP_CREDIT_PERC_MAX      PREV_APP_CREDIT_PERC_MAX
PREV_APP_CREDIT_PERC_MEAN    PREV_APP_CREDIT_PERC_MEAN
INSTAL_PAYMENT_PERC_MEAN      INSTAL_PAYMENT_PERC_MEAN
INSTAL_PAYMENT_PERC_SUM        INSTAL_PAYMENT_PERC_SUM
dtype: object

Indices des lignes avec des valeurs infinies :
[5687, 60477, 79077, 89018, 98509, 126768, 128791, 140426, 152087, 167136, 199103, 201086, 236164, 238381, 272829, 277962, 287300, 292852, 305373]

Reste-t-il des valeurs infinies ?
False

Reste-t-il des valeurs infinies après remplacement ?
False


In [15]:
# Coût d'un faux positif et d'un faux négatif
cost_fp = 1  
cost_fn = 10  

# Encodage des variables catégorielles avec OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
df_encoded = pd.DataFrame(ordinal_encoder.fit_transform(df), columns=df.columns)

# Vérification des colonnes non numériques
non_numeric_cols = df_encoded.select_dtypes(exclude=[np.number]).columns
df_encoded.drop(columns=non_numeric_cols, inplace=True)

# Gestion des valeurs manquantes avec SimpleImputer
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(df_encoded.drop(columns=['TARGET'])), columns=df_encoded.drop(columns=['TARGET']).columns)

# Séparation des caractéristiques et la cible
y = df_encoded['TARGET']

# Séparation des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Échantillonner les données
sample_size = 0.1 
X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, test_size=(1 - sample_size), random_state=42)
X_test_sample, _, y_test_sample, _ = train_test_split(X_test, y_test, test_size=(1 - sample_size), random_state=42)

# Afficher les colonnes utilisées pour l'entraînement
print("Colonnes utilisées pour l'entraînement :")
print(X_train_sample.columns.tolist())

# Sauvegarder les colonnes utilisées pour l'entraînement dans un fichier
columns_file = "columns_used_for_training.txt"
with open(columns_file, 'w') as f:
    for col in X_train_sample.columns.tolist():
        f.write(f"{col}\n")

# Adresse de MLflow
mlflow_url = "http://127.0.0.1:5000"

# Configurer l'expérience MLflow
mlflow.set_tracking_uri(mlflow_url)
mlflow.set_experiment("Credit Scoring Experiment")

# Définir le modèle RandomForestClassifier avec ses paramètres
model = RandomForestClassifier(random_state=42)  
params = {'n_estimators': [50, 100], 'max_depth': [3, 6, 9]}  

# Entraînement du modèle RandomForestClassifier obtention des meilleurs paramètres
with mlflow.start_run(run_name="RandomForest"):
    print("Entraînement du modèle RandomForest...")

    grid_search = GridSearchCV(estimator=model, param_grid=params, scoring='f1', cv=3)
    grid_search.fit(X_train_sample, y_train_sample)

    # Loguer les paramètres dans MLflow
    mlflow.log_params(grid_search.best_params_)

    # Affichage des meilleurs paramètres et le score F1
    print(f"Meilleurs paramètres pour RandomForest: {grid_search.best_params_}")
    print(f"Score F1 moyen sur le jeu de validation: {grid_search.best_score_:.3f}")

    # Utilisation du meilleur modèle pour les prédictions
    best_model = grid_search.best_estimator_
    y_prob = best_model.predict_proba(X_test_sample)[:, 1]

    # Évaluation du modèle avec le meilleur seuil pour minimiser le coût métier
    thresholds = np.linspace(0, 1, 100)
    costs = []
    for threshold in thresholds:
        y_pred_thresholded = (y_prob > threshold).astype(int)
        fp = np.sum((y_pred_thresholded == 1) & (y_test_sample == 0)) * cost_fp
        fn = np.sum((y_pred_thresholded == 0) & (y_test_sample == 1)) * cost_fn
        total_cost = fp + fn
        costs.append(total_cost)

    best_threshold = thresholds[np.argmin(costs)]
    print(f"Meilleur seuil pour minimiser le coût métier avec RandomForest: {best_threshold}")

    y_pred_best_threshold = (y_prob > best_threshold).astype(int)

    # Affichage du rapport de classification et de la matrice de confusion
    classification_report_str = classification_report(y_test_sample, y_pred_best_threshold)
    confusion_matrix_str = confusion_matrix(y_test_sample, y_pred_best_threshold)

    print(f"Rapport de classification avec le meilleur seuil pour RandomForest:")
    print(classification_report_str)

    print(f"Matrice de confusion avec le meilleur seuil pour RandomForest:")
    print(confusion_matrix_str)
    print()

    # Loguer les métriques et les résultats dans MLflow
    mlflow.log_metric("best_score_f1", grid_search.best_score_)
    mlflow.log_metric("best_threshold", best_threshold)
    mlflow.log_text(classification_report_str, "classification_report.txt")
    mlflow.log_text(str(confusion_matrix_str), "confusion_matrix.txt")

    # Loguer le modèle
    model_path = 'model/random_forest_model.pkl'
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    mlflow.sklearn.log_model(best_model, "model")

    # Sauvegarder le modèle dans un fichier .pkl
    joblib.dump(best_model, model_path)

Colonnes utilisées pour l'entraînement :
['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AV

c:\Users\paulm\anaconda3\envs\notebook-6.5.7\Lib\site-packages\_distutils_hack\__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\paulm\anaconda3\envs\notebook-6.5.7\Lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [7]:
# Adresse de MLflow
mlflow_url = "http://127.0.0.1:5000"

cost_fp = 1  # Coût d'un faux positif
cost_fn = 10  # Coût d'un faux négatif (10 fois le coût d'un FP)

# Vérification des colonnes non numériques
non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns
df.drop(columns=non_numeric_cols, inplace=True)

# Encodage des variables catégorielles avec OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
df_encoded = pd.DataFrame(ordinal_encoder.fit_transform(df), columns=df.columns)

# Gestion des valeurs manquantes avec SimpleImputer
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(df_encoded.drop(columns=['TARGET'])), columns=df_encoded.drop(columns=['TARGET']).columns)

# Séparation des caractéristiques et la cible
y = df_encoded['TARGET']

# Séparation des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Définir les modèles à tester avec leurs paramètres
models = [
    ('Random Forest', RandomForestClassifier(class_weight='balanced', random_state=42))
]

params = {
    'Random Forest': {'n_estimators': [50, 100], 'max_depth': [None, 10, 20]}
}

# Configurer l'expérience MLflow
mlflow.set_tracking_uri(mlflow_url)
mlflow.set_experiment("Credit Scoring Experiment")

# Échantillonner les données
sample_size = 0.1  # Utilisation de 10% des données pour un échantillonnage rapide
X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, test_size=(1 - sample_size), random_state=42)
X_test_sample, _, y_test_sample, _ = train_test_split(X_test, y_test, test_size=(1 - sample_size), random_state=42)

# Test des modèles pour trouver les meilleurs paramètres
for name, model in models:
    with mlflow.start_run(run_name=name):
        print(f"Entraînement du modèle {name}...")

        grid_search = GridSearchCV(estimator=model, param_grid=params[name], scoring='f1', cv=3)
        grid_search.fit(X_train_sample, y_train_sample)

        # Loguer les paramètres dans MLflow
        mlflow.log_params(grid_search.best_params_)

        # Afficher les meilleurs paramètres et le score F1
        print(f"Meilleurs paramètres pour {name}: {grid_search.best_params_}")
        print(f"Score F1 moyen sur le jeu de validation: {grid_search.best_score_:.3f}")

        # Utilisation du meilleur modèle pour les prédictions
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test_sample)

        # Évaluer le modèle avec le meilleur seuil pour minimiser le coût métier
        y_prob = best_model.predict_proba(X_test_sample)[:, 1]
        thresholds = np.linspace(0, 1, 100)
        costs = []
        for threshold in thresholds:
            y_pred_thresholded = (y_prob > threshold).astype(int)
            fp = np.sum((y_pred_thresholded == 1) & (y_test_sample == 0)) * cost_fp
            fn = np.sum((y_pred_thresholded == 0) & (y_test_sample == 1)) * cost_fn
            total_cost = fp + fn
            costs.append(total_cost)

        best_threshold = thresholds[np.argmin(costs)]
        print(f"Meilleur seuil pour minimiser le coût métier avec {name}: {best_threshold}")

        y_pred_best_threshold = (y_prob > best_threshold).astype(int)

        # Affichage du rapport de classification et la matrice de confusion
        classification_report_str = classification_report(y_test_sample, y_pred_best_threshold)
        confusion_matrix_str = confusion_matrix(y_test_sample, y_pred_best_threshold)

        print(f"Rapport de classification avec le meilleur seuil pour {name}:")
        print(classification_report_str)

        print(f"Matrice de confusion avec le meilleur seuil pour {name}:")
        print(confusion_matrix_str)
        print()

        # Loguer les métriques et les résultats dans MLflow
        mlflow.log_metric("best_score_f1", grid_search.best_score_)
        mlflow.log_metric("best_threshold", best_threshold)
        mlflow.log_text(classification_report_str, "classification_report.txt")
        mlflow.log_text(str(confusion_matrix_str), "confusion_matrix.txt")

        # Loguer le modèle
        mlflow.sklearn.log_model(best_model, "model")


Entraînement du modèle Random Forest...
Meilleurs paramètres pour Random Forest: {'max_depth': 10, 'n_estimators': 100}
Score F1 moyen sur le jeu de validation: 0.246
Meilleur seuil pour minimiser le coût métier avec Random Forest: 0.38383838383838387
Rapport de classification avec le meilleur seuil pour Random Forest:
              precision    recall  f1-score   support

         0.0       0.96      0.72      0.82      5663
         1.0       0.16      0.62      0.25       486

    accuracy                           0.71      6149
   macro avg       0.56      0.67      0.54      6149
weighted avg       0.89      0.71      0.78      6149

Matrice de confusion avec le meilleur seuil pour Random Forest:
[[4082 1581]
 [ 187  299]]

